In [1]:
from bokeh.models.sources import ColumnDataSource
from bokeh.plotting import figure
from bokeh.layouts import layout, row
from bokeh.io import output_notebook, show, push_notebook
from ipywidgets import Play, IntSlider, HBox, jslink, Label
import numpy as np

output_notebook()

Loading BokehJS ...

In [2]:
np.random.seed(0)
data = {"x": np.random.rand(10),
        "y": np.random.rand(10),
        "size": 10*np.random.rand(10, 100)}

class DummyChange:
    def __init__(self, val):
        self.new = val

In [7]:
class Streamer:
    def __init__(self):
        self._data = None
        self._n_points = 0
        self._cds = ColumnDataSource(data=dict(x=[], y=[], size=[]))
        self._layout = layout([])
        self._pw = Play(value=0,
                        min=0,
                        max=1,
                        step=1,
                        description="",
                        disabled=True)
        self._is = IntSlider(value=0,
                             min=0,
                             max=1,
                             step=1,
                             disabled=True)
        self._t = Label(value="0")
        jslink((self._is, "value"), (self._pw, "value"))
        self._pw.observe(self._animate, names="value")        

        self._handle = show(self._layout, notebook_handle=True)
        display(HBox([self._pw, self._is, self._t]))

    def _animate(self, change):
        frame = change.new
        self._cds.stream(new_data=dict(x=self._data["x"], # no change
                                       y=self._data["y"], # no change
                                       size=self._data["size"][:, frame]),
                         rollover=self._n_points)
        push_notebook(handle=self._handle)
        self._t.value = f"frame {frame}, median size: {np.median(self._cds.data['size']):0.3f}"

    def set_data(self, data):
        self._data = data
        self._n_points = data["x"].size

        fig = figure(plot_width=800, plot_height=400)
        fig.scatter("x", "y", size="size", source=self._cds)
        self._layout.children = [row([fig])]

        # update player/slider widget values
        self._is.disabled = self._pw.disabled = False
        self._is.max = self._pw.max = self._data["size"].shape[1]

        self._animate(DummyChange(0))

s = Streamer()
s.set_data(data)

In [27]:
print(s._handle.doc.to_json_string())

{"roots":{"references":[{"attributes":{"children":[{"id":"1735","type":"Row"}]},"id":"1660","type":"Column"}],"root_ids":["1660"]},"title":"Bokeh Application","version":"1.3.4"}


In [19]:
print(s2._handle.doc.to_json_string())

{"roots":{"references":[{"attributes":{"children":[{"id":"1838","type":"Row"}]},"id":"1839","type":"Column"},{"attributes":{"callback":null},"id":"1804","type":"DataRange1d"},{"attributes":{},"id":"1821","type":"WheelZoomTool"},{"attributes":{"source":{"id":"1800","type":"ColumnDataSource"}},"id":"1837","type":"CDSView"},{"attributes":{},"id":"1887","type":"BasicTickFormatter"},{"attributes":{"dimension":1,"ticker":{"id":"1816","type":"BasicTicker"}},"id":"1819","type":"Grid"},{"attributes":{},"id":"1824","type":"ResetTool"},{"attributes":{},"id":"1825","type":"HelpTool"},{"attributes":{"fill_color":{"value":"#1f77b4"},"line_color":{"value":"#1f77b4"},"size":{"field":"size","units":"screen"},"x":{"field":"x"},"y":{"field":"y"}},"id":"1834","type":"Scatter"},{"attributes":{"callback":null,"data":{"size":{"__ndarray__":"8RsvooaSI0AkYEyUmgIdQCISACrGKBlAorePxFCgBUAckW8aMXwjQKvIvWVDvxVA4j57wMQmDECRDPEwdgoeQGQ0uTjOgRNA0YLMGfJKGkA=","dtype":"float64","shape":[10]},"x":{"__ndarray__":"qBJfVuGP

In [8]:
class Streamer2:
    def __init__(self, data):
        self._data = data
        self._n_points = data["x"].size
        self._cds = ColumnDataSource(data=dict(x=[], y=[], size=[]))

        fig = figure(plot_width=800, plot_height=400)
        fig.scatter("x", "y", size="size", source=self._cds)

        self._layout = layout(row([fig]))
        self._pw = Play(value=0,
                        min=0,
                        max=self._data["size"].shape[1],
                        step=1,
                        description="",
                        disabled=False)
        self._is = IntSlider(value=0,
                             min=0,
                             max=self._data["size"].shape[1],
                             step=1,
                             disabled=False)
        self._t = Label(value="0")
        jslink((self._is, "value"), (self._pw, "value"))
        self._pw.observe(self._animate, names="value")        

        self._handle = show(self._layout, notebook_handle=True)
        display(HBox([self._pw, self._is, self._t]))
        
        self._animate(DummyChange(0))

    def _animate(self, change):
        frame = change.new
        self._cds.stream(new_data=dict(x=self._data["x"], # no change
                                       y=self._data["y"], # no change
                                       size=self._data["size"][:, frame]),
                         rollover=self._n_points)
        push_notebook(handle=self._handle)
        self._t.value = f"frame {frame}, median size: {np.median(self._cds.data['size']):0.3f}"

s2 = Streamer2(data)